In [1]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
def getForums(site_url):
    col=['forum','subforum','forum_url']
    top_df = pd.DataFrame(columns=col)
    r = re.get(site_url)
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    for element in soup.find_all('td', {"class": "alt1Active"}):
        forum_title_div = element.div
        forum_link = forum_title_div.a
        forum_url = site_url + forum_link['href']
        if('.com' in forum_link['href']):
            forum_url = forum_link['href']
        forum_title = forum_link.strong.string
        subforum = ''

        row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace

        for child in element.find_all('div', {"style" : "margin-top:6px"}):
            if child.find('strong') != None: #has a Sub-Forums section
                for subforum_link in child.find_all('a'):
                    subforum = subforum_link.contents[0]
                    forum_url = site_url + subforum_link['href']
                    if('.com' in subforum_link['href']):
                        forum_url = subforum_link['href']
                    row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
                    if(len(top_df)==0):
                        top_df = row
                    else:
                        top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return top_df

In [4]:
site_url = 'https://forums.hardwarezone.com.sg'
hwz = getForums(site_url)
print(len(hwz))
hwz

119


,forum,subforum,forum_url
0,Tech Show Central (IT Show 2018),,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/
1,Hardware Clinic,,https://forums.hardwarezone.com.sg/hardware-clinic-2/
2,Hardware Clinic,CORBELL eCustomer Service Center,https://forums.hardwarezone.com.sg/corbell-ecustomer-service-center-166/
3,Hardware Clinic,CONVERGENT eCustomer Service Center,https://forums.hardwarezone.com.sg/convergent-ecustomer-service-center-184/
4,Hardware Clinic,ASUS eCustomer Service Center,https://forums.hardwarezone.com.sg/asus-ecustomer-service-center-336/
5,Hardware Clinic,Avertek eCustomer Service Center,https://forums.hardwarezone.com.sg/avertek-ecustomer-service-center-388/
6,Hardware Clinic,Hardware Clinic Knowledge Base,https://forums.hardwarezone.com.sg/hardware-clinic-knowledge-base-33/
7,Hardware Clinic,Personal Projects and Reviews Den,https://forums.hardwarezone.com.sg/personal-projects-reviews-den-72/
8,Notebook Clinic,,https://forums.hardwarezone.com.sg/notebook-clinic-77/
9,Notebook Clinic,Gaming / Entertainment Class Notebook Hub,https://forums.hardwarezone.com.sg/gaming-entertainment-class-notebook-hub-293/


Build a table of forums and subforums

In [5]:
def getThreads(forum_url):
    lastForumPage = False
    forum_cols = ['forum_url', 'thread', 'thread_url']
    forum_df = pd.DataFrame(columns=forum_cols)
    forum_page_url = forum_url
    while(not lastForumPage):
        r2 = re.get(forum_page_url)
        forum_page = r2.text
        forum_page_soup = BeautifulSoup(forum_page, 'html.parser')

        if (forum_page_soup.find('a', text='Next ›') == None):
            lastForumPage = True
        else:
            forum_page_url = site_url + forum_page_soup.find('a', text='Next ›')['href']

        for element in forum_page_soup.find_all('a', {"id": lambda x: x and 'thread_title_' in x}):
            row = pd.DataFrame([[forum_url, element.contents[0], site_url + element['href']]], columns=forum_cols)
            if(len(forum_df)==0):
                forum_df = row
            else:
                forum_df = forum_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return forum_df

In [6]:
parenting = getThreads('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/')
print(len(parenting))
parenting

155


,forum_url,thread,thread_url
0,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Welcome to HWZ's Parenting, Kids & Early Learning forum",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/welcome-hwzs-parenting-kids-early-learning-forum-5684416.html
1,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Milk powder,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/milk-powder-5773651.html
2,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,2017 Secondary School Ranking in Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/2017-secondary-school-ranking-singapore-5818040.html
3,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Anyone went thru abortion before?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/anyone-went-thru-abortion-before-5779121.html
4,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,I suspect my child has ADHD,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/i-suspect-my-child-has-adhd-5816748.html
5,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Do u lose your temper when teaching your child?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/do-u-lose-your-temper-when-teaching-your-child-5785587.html
6,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Cry when feeding,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/cry-when-feeding-5723877.html
7,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Newborn, nanny, infant care or maid after maternity leave?",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/newborn-nanny-infant-care-maid-after-maternity-leave-5805042.html
8,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Selling Obscene Films and Used Panties Can Be Illegal In Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/selling-obscene-films-used-panties-can-illegal-singapore-5815478.html
9,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Potty training,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/potty-training-5813079.html


Get threads in a forum/sub-forum - all pages

In [7]:
def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['thread_url', 'userid', 'timestamp', 'post_text'] #, 'post_number', 'post_order', 'first_quote_post_number', 'likes_userid'
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url

    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', text='Next ›') == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', text='Next ›')['href']

        thread_page_posts = thread_page_soup.find('div', {'id': 'posts'})
        
        try: 
            for post in thread_page_posts.find_all('div', {'class': 'post-wrapper'}):
                userid_url = post.find('a', {'class': 'bigusername'})['href']
                userid = ''.join(filter(lambda x: x.isdigit(), userid_url))

                datetime_raw = post.find('a', {'name': lambda x: x and x.find('post') == 0}).nextSibling.strip()
                date_list = datetime_raw.split(',')[0].split('-')
                iso_date = '-'.join(list(reversed(date_list)))
                hour = int(datetime_raw.split(' ')[1][0:2])
                if(datetime_raw.split(' ')[2] == 'PM' and hour < 12):
                    hour += 12
                hour_str = str(hour)
                if(hour < 10):
                    hour_str = '0' + str(hour)
                minute = datetime_raw.split(':')[1][0:2]
                iso_datetime = iso_date + 'T' + hour_str + ':' + minute

                post_text = ""
                try:
                    post_text = post.find('div', {'class': 'post_message'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass

                row = pd.DataFrame([[thread_url, userid, iso_datetime, post_text]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[thread_url, "", "", ""]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return thread_df

In [8]:
pointofchildren = getPosts('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html')
print(len(pointofchildren))
pointofchildren

232


,thread_url,userid,timestamp,post_text
0,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,642993,2017-04-21T10:09,"Most of the reasons for babies in SG is.......: -Because it is norm/Appease in laws -Accident/Abortion is EVIL -Insurance for old muahahaha -Love children much -Need a substitute doll to fulfill my dead dreams What else are there? other than point 4, really no point to keep wor. Unlike last time generation peepur..now cheaper to manage wealth than to rear babies since we are more educated ."
1,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,322359,2017-04-21T10:44,Can't finish all the money from dad
2,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,370175,2017-04-21T11:55,"In Singapore? No point. Unless you are filthy rich, there are utterly no justify-able reasons to have kids."
3,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,306175,2017-04-21T11:58,Ask your parents why bring you up
4,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,270705,2017-04-21T12:00,what is the point of ur existence...no point right? go kill urself
5,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,103529,2017-04-21T12:09,Tiagong government is bigger than your parents.
6,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,40851,2017-04-21T12:11,1) To create jobs by having more schools = teachers = janitor = etc 2) To make PAP happy
7,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,327041,2017-04-21T12:11,if your parents think the same way u won't even be here typing this nonsense
8,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,558576,2017-04-21T12:14,Point 4 still not enuff?
9,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,237992,2017-04-21T12:14,"In modern day context, anything but love children lol"


In [9]:
test = pd.merge(hwz, parenting, left_on = 'forum_url', right_on = 'forum_url', how = 'inner')
test

,forum,subforum,forum_url,thread,thread_url
0,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Welcome to HWZ's Parenting, Kids & Early Learning forum",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/welcome-hwzs-parenting-kids-early-learning-forum-5684416.html
1,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Milk powder,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/milk-powder-5773651.html
2,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,2017 Secondary School Ranking in Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/2017-secondary-school-ranking-singapore-5818040.html
3,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Anyone went thru abortion before?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/anyone-went-thru-abortion-before-5779121.html
4,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,I suspect my child has ADHD,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/i-suspect-my-child-has-adhd-5816748.html
5,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Do u lose your temper when teaching your child?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/do-u-lose-your-temper-when-teaching-your-child-5785587.html
6,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Cry when feeding,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/cry-when-feeding-5723877.html
7,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Newborn, nanny, infant care or maid after maternity leave?",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/newborn-nanny-infant-care-maid-after-maternity-leave-5805042.html
8,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Selling Obscene Films and Used Panties Can Be Illegal In Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/selling-obscene-films-used-panties-can-illegal-singapore-5815478.html
9,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Potty training,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/potty-training-5813079.html


In [ ]:
%%time
# hwz['forum_url'].apply(lambda x: pd.merge(hwz, getThreads(x), left_on = 'forum_url', right_on = 'forum_url', how = 'inner')
allThreads = pd.concat([getThreads(forum_url) for forum_url in hwz['forum_url']])

Forever and a day just to get the thread urls

Next time I should write these so that they write to file and pick up where they left off maybe

But threads get updated in real time, so maybe this has to be done at one go anyway.

In [10]:
allThreads_csv = os.path.join("data", "allthreads-2018-04-24T1300.csv")

In [ ]:
allThreads.to_csv(allThreads_csv, encoding='utf-8', index=False)

Better write that to file

In [11]:
allThreads_df = pd.read_csv(allThreads_csv)
print(len(allThreads_df))
allThreads_df.head()

156448


,forum_url,thread,thread_url
0,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,"**IT Show 2018 Show Preview, Tech Guides & Past Show Brochures**",https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%2A%2A-show-2018-show-preview-tech-guides-past-show-brochures%2A%2A-5788288.html
1,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,"**IT Show 2018 Tech Show Portal Launched! - Floor Plans, Brochures & More!**",https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%2A%2A-show-2018-tech-show-portal-launched-floor-plans-brochures-more-%2A%2A-5788382.html
2,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,Guide: How to buy an ideal Gaming Machine,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/guide-how-buy-ideal-gaming-machine-5764374.html
3,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,[Guide] Printers - Some things you might want to take note of,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%5Bguide%5D-printers-some-things-you-might-want-take-note-5366804.html
4,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,Tech Show Central Guidelines (MUST READ!),https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/tech-show-central-guidelines-must-read-5622593.html


In [12]:
def processQueryList(query_list, store=None):
    """Process a subset of the list of queries that needs to be made, storing posts in a dataframe"""
    if store is None:
        store = {}
    for url in query_list:
        #Store list of dataframes of posts from each thread into the dictionary
        store[str(url)] = getPosts(url)
    return store

In [13]:
from threading import Thread

def threadedApiCall(nthreads, master_query_list):
    store = {}
    threads = []
    sublistLength = len(master_query_list)//nthreads
    #split the master query list
    sublists = [master_query_list[x:x+sublistLength] for x in range(0,len(master_query_list), sublistLength)]
    for sublist in sublists:
        t = Thread(target=processQueryList, args=(sublist, store))
        threads.append(t)

    #start the threads
    [t.start() for t in threads]
    #wait for the threads to finish
    [t.join() for t in threads]
    
    return pd.concat(thread_posts for thread_posts in list(store.values())) 

In [ ]:
%%time
import string

for forum_url in hwz['forum_url']:
    forum_name = forum_url.split('/')[3].rstrip(string.digits).rstrip('-')
    forum_filepath = os.path.join('data', forum_name + "-2018-04-24T1300.csv")
    #Begin retrieving if file doesn't already exist
    if(!os.path.isfile(forum_filepath)):
        print('Start: ' + datetime.now().isoformat('T') + ' - ' + forum_name)
        forum_df = allThreads_df.loc[allThreads_df['forum_url'] == forum_url]
        #11 threads
        forum_pdf = threadedApiCall(10, forum_df['thread_url'])
        forum_pdf.to_csv(forum_filepath, encoding='utf-8', index=False)
        print('End: ' + datetime.now().isoformat('T') + ' - ' + forum_name)
    else:
        print(forum_name + ' CSV already generated.')

Since getPosts() handles requests threadwise already, we'll just try to parallelize thread retrieval.

Next we combine all the forum posts into one dataframe, then left join to the forum and site information